## 1\. Выбор правильного инструмента для задачи

### Ключевые различия

  * **`COPY` vs `\copy`:** Это самый частый источник путаницы.

      * **`COPY`** — это команда SQL, выполняемая **на сервере PostgreSQL**. Сервер должен иметь прямой доступ к файлу для чтения или записи. Это требует прав суперпользователя или членства в специальных ролях (`pg_read_server_files`, `pg_write_server_files`). Она невероятно быстра, так как работает на уровне файловой системы сервера.
      * **`\copy`** — это мета-команда клиента `psql`. Она выполняется **на клиентской машине**, где запущен `psql`. `psql` читает файл с вашего локального диска, передает данные на сервер через стандартное соединение, а затем сервер выполняет `COPY FROM STDIN`. Это универсальный и безопасный способ, не требующий прав на сервере.

  * **`COPY`/`\copy` vs `pg_dump`/`pg_restore`:** Разница в назначении.

      * **`COPY`/`\copy`** предназначены для **перемещения сырых данных** в одну таблицу или из нее. Они не знают ничего о структуре таблицы (кроме порядка столбцов), индексах, триггерах или внешних ключах. Это просто транспорт для данных.
      * **`pg_dump`/`pg_restore`** — это утилиты для **логического резервного копирования**. `pg_dump` извлекает не только данные, но и DDL-команды для воссоздания схемы: таблицы, представления, индексы, роли, права доступа и т.д. Это инструмент для создания полноценной, консистентной копии базы данных или ее частей.

  * **Текстовые форматы (CSV, TEXT) vs Бинарный формат vs Custom Format (`-Fc`)**

      * **Текстовые (CSV, TEXT):** Человекочитаемые, универсальные форматы. Идеальны для обмена данными с другими системами (Excel, Python/Pandas, BI-инструменты). CSV более гибок за счет опций (заголовки, разделители).
      * **Бинарный (`COPY BINARY`):** Максимально быстрый, но непереносимый формат. Он зависит от архитектуры машины и версии PostgreSQL. Используется для быстрой миграции данных между идентичными серверами.
      * **Custom Format (`-Fc` для `pg_dump`):** "Золотой стандарт" для бэкапов. Это сжатый, бинарный формат, который включает в себя и схему, и данные. Его главное преимущество — гибкость при восстановлении (`pg_restore`): можно восстанавливать базу параллельно, выбирать отдельные таблицы или только схему.

### Таблица-решение: Быстрый выбор инструмента

| Задача | Рекомендуемый инструмент | Формат | Пример команды и обоснование |
| :--- | :--- | :--- | :--- |
| **Выгрузить данные таблицы для анализа в Excel** | `\copy` | CSV | `\copy (SELECT * FROM sales) TO 'sales.csv' WITH CSV HEADER;`\<br\> *Клиентская команда, создает CSV с заголовками, идеально для Excel.* |
| **Быстро загрузить большой CSV-файл с сервера** | `COPY` | CSV | `COPY users FROM '/var/lib/postgresql/data/users.csv' WITH CSV;`\<br\> *Максимальная скорость, файл уже на сервере.* |
| **Создать полный бэкап базы данных для восстановления** | `pg_dump` | Custom (`-Fc`) | `pg_dump -Fc -U postgres mydb > mydb.dump`\<br\> *Сжатый, надежный формат, позволяет параллельное и выборочное восстановление.* |
| **Перенести схему (без данных) в другую БД** | `pg_dump` | Plain (`-p`) | `pg_dump -s -U postgres mydb > schema.sql`\<br\> *Создает текстовый SQL-файл, который легко читать и редактировать.* |
| **Восстановить одну таблицу из полного бэкапа** | `pg_restore` | Custom (`-Fc`) | `pg_restore -t my_table -d new_db mydb.dump`\<br\> *Главное преимущество формата `-Fc` — выборочное восстановление.* |

-----

## 2\. Фундамент: Команды COPY и \\copy

Это рабочие лошадки для ежедневных задач импорта/экспорта.

### Детальный разбор синтаксиса
Для терминала Windows следует удалить запись  `| STDOUT`   из команды
```sql
-- Экспорт данных ИЗ таблицы В файл
COPY table_name [(column1, column2, ...)] TO 'filename' WITH (options);

-- Импорт данных В таблицу ИЗ файла
COPY table_name [(column1, column2, ...)] FROM 'filename' WITH (options);
```

  * `table_name`: Целевая таблица.
  * `[(column1, ...)]`: Опциональный список столбцов. Позволяет указать, какие столбцы и в каком порядке выгружать/загружать.
  * `TO|FROM`: Направление операции.
  * `'filename'` : Источник или приемник. `'filename'` — путь к файлу **на сервере**.
  * `WITH (options)`: Блок для указания формата и его параметров.

### Форматы и опции (с примерами)

Предположим, у нас есть таблица:

```sql
CREATE TABLE employees (
    id INT PRIMARY KEY,
    full_name TEXT NOT NULL,
    department TEXT,
    salary NUMERIC(10, 2),
    hire_date DATE
);
```

#### **CSV (Comma-Separated Values)**

Самый гибкий текстовый формат.

  * `HEADER`: Указывает, что первая строка файла является заголовком. При импорте эта строка пропускается, при экспорте — добавляется.
  * `DELIMITER ','`: Задает разделитель полей (по умолчанию запятая). Можно использовать любой символ, например `';'` или `'|'`.
  * `QUOTE '"'`: Символ для обрамления значений, содержащих разделитель или спецсимволы.
  * `ESCAPE '\'`: Символ для экранирования символа кавычки внутри значения.
  * `NULL ''`: Указывает, как представлять `NULL`-значения в файле. По умолчанию это `\N`. Пустая строка (`''`) — частый выбор.

**Пример экспорта в CSV:**

```sql
\copy employees TO 'employees.csv' WITH (FORMAT CSV, HEADER, DELIMITER ';', NULL 'N/A');
-- или 
COPY employees TO 'D:\employee.csv' WITH (FORMAT CSV, HEADER, DELIMITER ';', NULL 'N/A');
```

*Результат в `employees.csv`:*

```csv
id;full_name;department;salary;hire_date
1;John Doe;Engineering;75000.00;2022-01-15
2;Jane Smith;Marketing;N/A;2023-03-10
```

**Пример импорта из CSV:**

```sql
-- Сначала очистим таблицу для чистоты эксперимента
TRUNCATE employees;

\copy employees FROM 'employees.csv' WITH (FORMAT CSV, HEADER, DELIMITER ';', NULL 'N/A');
```

#### **TEXT (Текстовый формат по умолчанию)**

Простой формат, где разделителем по умолчанию является символ табуляции (`\t`).

**Пример экспорта в TEXT:**

```sql
\copy employees TO 'employees.txt'; -- FORMAT TEXT можно опустить, это дефолт
```

*Результат в `employees.txt` (пробелы здесь — это табуляция):*

```
1   John Doe    Engineering 75000.00    2022-01-15
2   Jane Smith  Marketing   \N          2023-03-10
```

### Работа с подмножествами данных

Часто нужно выгрузить не всю таблицу.

  * **Выборочные столбцы и строки (Экспорт):** Используйте `COPY` с подзапросом `SELECT`.
    ```sql
    \copy (SELECT full_name, salary FROM employees WHERE department = 'Engineering') TO 'engineers.csv' WITH CSV HEADER;
    ```
  * **Выборочные столбцы (Импорт):** Укажите список столбцов после имени таблицы.
    ```sql
    -- CSV файл 'new_hires.csv' содержит только имя и зарплату
    -- full_name,salary
    -- "Alice Wonderland",60000.00

    -- Загружаем данные, указывая, в какие столбцы их класть. `id` будет сгенерирован, если это SERIAL.
    \copy employees(full_name, salary) FROM 'new_hires.csv' WITH CSV HEADER;
    ```

-----

## 3\. Профессиональная обработка ошибок и валидация данных

При работе с реальными данными ошибки неизбежны: неверный формат даты, нарушение ограничений и т.д. `COPY` по умолчанию работает по принципу "все или ничего" — одна ошибка, и вся операция откатывается.

### Стратегия промежуточной таблицы (Staging Table) — лучшая практика

Это самый надежный и гибкий подход к импорту "грязных" данных.

1.  **Создаем Staging-таблицу:** Структура этой таблицы повторяет структуру CSV-файла, но **все столбцы имеют тип `TEXT`**. Это гарантирует, что `COPY` примет любые данные без ошибок типов.

    ```sql
    CREATE TEMP TABLE employees_staging (
        id TEXT,
        full_name TEXT,
        department TEXT,
        salary TEXT,
        hire_date TEXT
    );
    ```

    *Использование `TEMP TABLE` удобно, так как таблица будет автоматически удалена в конце сессии.*

2.  **Загружаем сырые данные:** Выполняем `COPY` в эту промежуточную таблицу. Этот шаг практически никогда не падает.

    ```sql
    \copy employees_staging FROM 'dirty_data.csv' WITH CSV HEADER;
    ```

3.  **Валидация и очистка с помощью SQL:** Теперь, когда данные в базе, мы можем использовать всю мощь SQL для их проверки, очистки, преобразования и вставки в целевую таблицу.

    ```sql
    INSERT INTO employees (id, full_name, department, salary, hire_date)
    SELECT
        CAST(NULLIF(id, '') AS INT),
        full_name,
        NULLIF(department, 'N/A'), -- Превращаем 'N/A' в NULL
        CAST(REPLACE(salary, '$', '') AS NUMERIC), -- Убираем знаки валют
        TO_DATE(hire_date, 'MM/DD/YYYY') -- Преобразуем формат даты
    FROM
        employees_staging
    -- Отсеиваем строки с явными ошибками, которые не можем исправить
    WHERE
        id ~ '^[0-9]+$' -- Проверяем, что ID - это число
        AND full_name IS NOT NULL;
    ```

    **Почему это лучшая практика?**

      * **Изоляция ошибок:** `COPY` успешно завершается, а "плохие" строки остаются в `staging` таблице для анализа.
      * **Сложные преобразования:** Вы можете выполнять любую логику на SQL, которую невозможно выразить в опциях `COPY`.
      * **Аудит и отладка:** Легко найти и проанализировать строки, которые не прошли валидацию.

### Управление транзакциями

Для гарантии атомарности операции импорта, всегда оборачивайте `COPY` в транзакцию.

```sql
BEGIN;
TRUNCATE employees; -- Опционально, если нужно перезаписать данные
\copy employees FROM 'employees.csv' WITH CSV;
COMMIT;
-- Если на этапе \copy произойдет ошибка, транзакция автоматически прервется,
-- и TRUNCATE будет отменен (ROLLBACK). Таблица останется в исходном состоянии.
```

-----

## 4\. Оптимизация производительности для больших объемов данных

`COPY` — это самый быстрый способ загрузки данных в PostgreSQL. Давайте разберемся, как сделать его еще быстрее.

### Почему `COPY` быстрый?

`COPY` работает на низком уровне, минуя значительную часть накладных расходов SQL-парсера и планировщика. При определенных условиях (например, загрузка в пустую таблицу, созданную в той же транзакции) он может минимизировать запись в WAL (Write-Ahead Log), что дает колоссальный прирост скорости.

### Ключевые техники оптимизации

1.  **Обертывание в одну транзакцию:** Как уже упоминалось, `BEGIN; COPY ...; COMMIT;` не только обеспечивает атомарность, но и снижает накладные расходы на управление транзакциями для каждой строки.

2.  **Манипуляции с индексами и ограничениями (самый важный пункт\!):**

      * **Что делать:** Перед началом большого импорта (`COPY` на миллионы строк) **удалите все индексы (кроме Primary Key, если он не нужен для проверок), внешние ключи (Foreign Keys) и триггеры** с целевой таблицы.
      * **Почему это работает:**
          * **Индексы:** При вставке каждой строки PostgreSQL должен обновить *каждый* индекс на таблице. Это огромное количество случайных операций ввода-вывода, которые замедляют процесс в разы. Гораздо быстрее загрузить все данные в "голую" таблицу, а затем одной командой `CREATE INDEX` построить индекс. Эта операция выполняет одну последовательную читку таблицы и строит индекс гораздо эффективнее.
          * **Внешние ключи (FK):** При вставке каждой строки СУБД должна проверить наличие соответствующего ключа в связанной таблице. Это еще одна операция поиска на каждую строку. Проверка `ALTER TABLE ... VALIDATE CONSTRAINT` после загрузки всех данных делает это один раз для всей таблицы.
          * **Триггеры:** Отключение триггеров избавляет от накладных расходов на их запуск для каждой строки.
      * **Примерный воркфлоу:**
        ```sql
        BEGIN;
        -- 1. Удаляем индексы и FK
        DROP INDEX IF EXISTS idx_employees_department;
        ALTER TABLE employees DROP CONSTRAINT fk_department;

        -- 2. Загружаем данные
        COPY employees FROM 'massive_data.csv' WITH CSV;

        -- 3. Воссоздаем индексы и FK
        CREATE INDEX idx_employees_department ON employees(department);
        ALTER TABLE employees ADD CONSTRAINT fk_department FOREIGN KEY (...) REFERENCES ...;
        COMMIT;
        ```

3.  **Настройка памяти:** Перед воссозданием индексов увеличьте параметр `maintenance_work_mem` для вашей сессии. Это позволит PostgreSQL выделить больше оперативной памяти для операции построения индекса, что значительно ее ускорит.

    ```sql
    SET maintenance_work_mem = '2GB'; -- Установите значение в зависимости от доступной RAM
    CREATE INDEX ...;
    ```

4.  **Параллелизм:** Эта опция относится к `pg_restore`. При восстановлении из дампа формата `custom` (`-Fc`) используйте флаг `-j` (или `--jobs`), чтобы запустить несколько процессов восстановления параллельно. Это может кратно ускорить процесс на многоядерных серверах.

    ```bash
    pg_restore -j 8 -d new_db mydb.dump
    ```

-----

## 5\. Инструменты логического бэкапа: pg\_dump и pg\_restore

Эти утилиты — ваш швейцарский нож для миграций, бэкапов и клонирования.

### Сравнение форматов (`-F`)

  * **`plain` (`-Fp`, по умолчанию):** Создает большой текстовый `.sql` файл.
      * **Плюсы:** Человекочитаемый, можно редактировать, легко отслеживать изменения в Git.
      * **Минусы:** Огромный размер, медленное восстановление (только в один поток), нет гибкости.
  * **`tar` (`-Ft`):** Создает `.tar` архив.
      * **Плюсы:** Позволяет выборочно восстанавливать объекты.
      * **Минусы:** Не сжат, во многом уступает формату `custom`.
  * **`custom` (`-Fc`):** **Рекомендуемый формат для большинства задач.**
      * **Плюсы:** Сжат по умолчанию, включает и схему, и данные, позволяет **параллельное восстановление** (`-j`), позволяет **выборочно восстанавливать** таблицы, схемы или только данные/схему.
      * **Минусы:** Не человекочитаемый, требует `pg_restore` для использования.

### Сценарии использования (с примерами команд)

  * **Дамп всей БД в формате custom:**
    ```bash
    pg_dump -U postgres -h localhost -Fc --verbose -f my_database.dump my_database
    ```
  * **Восстановление всей БД из дампа:**
    ```bash
    # Сначала нужно создать пустую базу
    createdb -U postgres new_database
    pg_restore -U postgres -h localhost -d new_database -j 8 my_database.dump
    ```
  * **Дамп только схемы (структуры):**
    ```bash
    pg_dump -U postgres -s -f schema.sql my_database
    ```
  * **Дамп только данных (без схемы):**
    ```bash
    pg_dump -U postgres -a -f data.sql my_database
    ```
  * **Выборочное восстановление (Selective Restore):**
      * Сначала посмотрим содержимое дампа:
        ```bash
        pg_restore -l my_database.dump > dump_contents.txt
        ```
      * Затем восстановим только таблицу `public.employees`:
        ```bash
        pg_restore -U postgres -d new_database -t employees my_database.dump
        ```
  * **Дамп всего кластера (`pg_dumpall`):**
    Эта утилита нужна для бэкапа **глобальных объектов**, которые не принадлежат конкретной базе данных: ролей (пользователей) и табличных пространств. Стандартный воркфлоу для полного бэкапа кластера:
    ```bash
    # 1. Дампим глобальные объекты
    pg_dumpall -U postgres --globals-only > globals.sql

    # 2. Дампим каждую базу данных отдельно (можно в цикле)
    pg_dump -U postgres -Fc -f db1.dump db1
    pg_dump -U postgres -Fc -f db2.dump db2
    ```

-----

## 6\. Интеграция, автоматизация и экосистема

### `psql` в скриптах

`psql` — мощный инструмент для автоматизации.

  * `-c "command"`: Выполнить одну команду и выйти.
    ```bash
    psql -U postgres -d mydb -c "SELECT count(*) FROM users;"
    ```
  * `-f script.sql`: Выполнить команды из файла.
    ```bash
    psql -U postgres -d mydb -f /path/to/my_script.sql
    ```

### Пример Bash-скрипта для автоматизации бэкапа

Этот скрипт делает полный дамп схемы и выгружает ключевую таблицу в CSV для архивации.

```bash
#!/bin/bash
set -e # Прервать выполнение при любой ошибке

# --- Настройки ---
DB_USER="postgres"
DB_NAME="production_db"
BACKUP_DIR="/var/backups/postgresql"
DATE=$(date +"%Y-%m-%d_%H%M")

# --- Создание директории бэкапа ---
mkdir -p "$BACKUP_DIR"

# --- 1. Дамп схемы ---
echo "Dumping schema..."
pg_dump -U "$DB_USER" -d "$DB_NAME" --schema-only -f "$BACKUP_DIR/schema-$DATE.sql"

# --- 2. Выгрузка критически важных данных в CSV ---
echo "Exporting critical 'transactions' table to CSV..."
psql -U "$DB_USER" -d "$DB_NAME" -c "\copy (SELECT * FROM transactions WHERE created_at > NOW() - INTERVAL '1 day') TO '$BACKUP_DIR/transactions-$DATE.csv' WITH CSV HEADER"

# --- 3. Архивирование ---
echo "Creating archive..."
tar -czf "$BACKUP_DIR/backup-$DATE.tar.gz" -C "$BACKUP_DIR" "schema-$DATE.sql" "transactions-$DATE.csv"

# --- 4. Очистка временных файлов ---
rm "$BACKUP_DIR/schema-$DATE.sql" "$BACKUP_DIR/transactions-$DATE.csv"

# --- 5. Удаление старых бэкапов (старше 7 дней) ---
find "$BACKUP_DIR" -type f -name "*.tar.gz" -mtime +7 -delete

echo "Backup complete: $BACKUP_DIR/backup-$DATE.tar.gz"
```

### Сторонние инструменты и языки

  * **Python:**
      * **`psycopg2`:** Для максимальной производительности используйте `cursor.copy_expert()` или `cursor.copy_from()`. Это позволяет потоково передавать данные из вашего приложения прямо в `COPY STDIN`, минуя создание промежуточных файлов.
      * **`pandas` + `sqlalchemy`:** Для ETL и аналитических задач `DataFrame.to_sql(method='multi')` является удобной оберткой, но для очень больших объемов прямой `COPY` через `psycopg2` будет быстрее.
  * **Графические инструменты:** `pgAdmin` и `DBeaver` имеют удобные GUI для импорта/экспорта. Важно понимать, что под капотом они генерируют и выполняют те же самые команды `COPY`/`\copy`.

-----

## 7\. Безопасность и итоги

### Безопасность, кодировка и документирование

  * **Документирование:** Всегда сохраняйте команду `pg_dump`, которой был сделан бэкап. Это поможет избежать ошибок при восстановлении (например, забыть про флаг `--no-owner`).
  * **Шифрование:** Если ваши дампы содержат чувствительные данные (PII), шифруйте их перед отправкой в хранилище (например, S3).
    ```bash
    pg_dump -Fc ... | gpg -c -o backup.dump.gpg
    ```
  * **Кодировка (Encoding):** 99% проблем с кириллицей и другими не-латинскими символами решаются правильной настройкой кодировки. Убедитесь, что ваша база создана в `UTF8`. При работе с `psql` переменная окружения `PGCLIENTENCODING` должна быть установлена в `UTF8`. При `COPY` можно явно указать кодировку: `COPY ... FROM 'file.csv' WITH (ENCODING 'UTF8')`.

### Чек-лист для импорта больших данных в Production

Перед тем как запустить импорт гигабайтов данных на боевом сервере, пройдитесь по этому списку:

1.  [ ] **Бэкап:** Сделан ли свежий, проверенный бэкап целевой базы данных?
2.  [ ] **Место на диске:** Достаточно ли места для самих данных, WAL-файлов и для построения новых индексов? (Построение индекса требует дополнительного места, сопоставимого с размером самого индекса).
3.  [ ] **Транзакция:** Вся операция импорта обернута в `BEGIN...COMMIT`?
4.  [ ] **Отключение:** Отключены (или удалены) ли все некритичные индексы, внешние ключи и триггеры на целевой таблице?
5.  [ ] **Память:** Увеличен ли `maintenance_work_mem` для сессии перед созданием индексов?
6.  [ ] **Мониторинг:** Настроен ли мониторинг нагрузки на CPU, I/O и потребления дискового пространства на время операции?
7.  [ ] **План отката:** Что вы будете делать, если что-то пойдет не так?

Надеюсь, это руководство станет вашим верным спутником в мире PostgreSQL. Удачи\! 🚀